In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path
import pandas as pd

SALARY_LEMMA_NAME = "hh_vacancies_salary_with_lemma.parquet"
DRIVE_DIR = Path("/content/drive/MyDrive/ColabNotebooks/FinalProject/ProcessedDatasets")

salary_path = DRIVE_DIR / SALARY_LEMMA_NAME

if salary_path.exists():
    df_salary = pd.read_parquet(salary_path)
else:
    from google.colab import files
    uploaded = files.upload()
    df_salary = pd.read_parquet(SALARY_LEMMA_NAME)

print("Размер датасета:", df_salary.shape)

Размер датасета: (14131, 31)


In [3]:
df_cluster = df_salary.copy()

df_cluster["desc_lemma_len"] = df_cluster["description_lemma"].str.len()

MIN_LEN = 200
df_cluster = df_cluster[df_cluster["desc_lemma_len"] >= MIN_LEN].copy()

print(df_cluster["desc_lemma_len"].describe())
print("Размер после фильтра:", df_cluster.shape)

count    14006.000000
mean      1306.677924
std        691.152788
min        200.000000
25%        805.000000
50%       1204.500000
75%       1670.000000
max       8395.000000
Name: desc_lemma_len, dtype: float64
Размер после фильтра: (14006, 32)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

custom_stopwords = [
    "год", "компания", "работа", "обязанность",
    "требование", "умение", "навык", "опыт",
    "высокий", "доход", "заработный", "зарплата",
    "оплата", "вознаграждение"
]

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=20,
    max_df=0.6,
    sublinear_tf=True,
    stop_words=custom_stopwords
)

X = vectorizer.fit_transform(df_cluster["description_lemma"])
terms = np.array(vectorizer.get_feature_names_out())

print("TF-IDF shape:", X.shape)

TF-IDF shape: (14006, 16058)


In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

SAMPLE_SIZE = 3000

np.random.seed(42)
sample_idx = np.random.choice(X.shape[0], SAMPLE_SIZE, replace=False)
X_sample = X[sample_idx]

ks = [5, 6, 7, 8, 9, 10]
sil_scores = []

for k in ks:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_sample)
    sil = silhouette_score(X_sample, labels)
    sil_scores.append(sil)
    print(f"k={k}: silhouette={sil:.4f}")

best_k = ks[int(np.argmax(sil_scores))]
print("\nBest k by silhouette:", best_k)

k=5: silhouette=0.0061
k=6: silhouette=0.0071
k=7: silhouette=0.0093
k=8: silhouette=0.0096
k=9: silhouette=0.0102
k=10: silhouette=0.0104

Best k by silhouette: 10


In [6]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

k = 10
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df_cluster["cluster"] = kmeans.fit_predict(X)

print("Распределение по кластерам (n):")
display(df_cluster["cluster"].value_counts().sort_index().to_frame("count"))

terms = np.array(vectorizer.get_feature_names_out())
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

top_terms_per_cluster = []
TOP_N = 12

for i in range(k):
    top_terms = [terms[ind] for ind in order_centroids[i, :TOP_N]]
    top_terms_per_cluster.append({
        "cluster": i,
        "count": int((df_cluster["cluster"] == i).sum()),
        "top_terms": ", ".join(top_terms)
    })

top_terms_df = pd.DataFrame(top_terms_per_cluster).sort_values("cluster")
print("\nТоп-термы по кластерам:")
display(top_terms_df)

cluster_salary = (df_cluster
                  .groupby("cluster")["salary_mid"]
                  .agg(count="count", mean="mean", median="median")
                  .sort_values("median", ascending=False))

print("\nЗарплаты по кластерам (sorted by median):")
display(cluster_salary)

Распределение по кластерам (n):


,count
cluster,
0,1506
1,1247
2,2701
3,2094
4,1326
5,1767
6,114
7,838
8,1931



Топ-термы по кластерам:


,cluster,count,top_terms
0,0,1506,"продажа, клиент, переговоры, база, менеджер, к..."
1,1,1247,"объект, технический, документация, строительны..."
2,2,2701,"день, автомобиль, место, месяц, плата, рабочий..."
3,3,2094,"документ, подготовка, организация, ведение, зн..."
4,4,1326,"смена, гость, заказ, час, питание, бесплатный,..."
5,5,1767,"разработка, проект, команда, понимание, задача..."
6,6,114,"неполный рабочий, неполный, отдых, продавец ка..."
7,7,838,"пациент, медицинский, клиника, специальность, ..."
8,8,1931,"наш, клиент, мы, что, команда, который, если, ..."
9,9,482,"бухгалтерский, учёт, бухгалтер, налоговый, док..."



Зарплаты по кластерам (sorted by median):


,count,mean,median
cluster,,,
1,1247,158290.852322,150000.0
0,1506,160123.140491,147247.5
5,1767,158049.614233,140000.0
9,482,130828.533548,120000.0
4,1326,115450.352123,100000.0
3,2094,109669.195826,100000.0
8,1931,117727.344570,100000.0
7,838,118111.232757,100000.0
2,2701,111811.004235,95700.0


In [7]:
q25 = df_cluster["salary_mid"].quantile(0.25)
q75 = df_cluster["salary_mid"].quantile(0.75)

print("Q25:", q25)
print("Q75:", q75)

def salary_group_func(x):
    if x <= q25:
        return "low"
    elif x >= q75:
        return "high"
    else:
        return "mid"

df_cluster["salary_group"] = df_cluster["salary_mid"].apply(salary_group_func)

print(df_cluster["salary_group"].value_counts())

Q25: 80000.0
Q75: 152250.0
salary_group
mid     6912
low     3559
high    3535
Name: count, dtype: int64


In [8]:
pd.crosstab(
    df_cluster["cluster"],
    df_cluster["salary_group"],
    normalize="index"
).round(3)

salary_group,high,low,mid
cluster,,,
0,0.422,0.108,0.471
1,0.456,0.067,0.477
2,0.165,0.340,0.495
3,0.134,0.301,0.565
4,0.176,0.287,0.537
5,0.418,0.178,0.405
6,0.000,0.658,0.342
7,0.198,0.284,0.518
8,0.186,0.356,0.458


**Методика**

Для выявления естественной структуры вакансий была проведена кластеризация текстов описаний без использования информации о заработной плате (unsupervised learning).

В качестве текстового представления использовалась TF-IDF-векторизация лемматизированных описаний вакансий (униграммы и биграммы).
Кластеризация выполнена методом KMeans (k = 10, выбран на основе анализа silhouette score).

Перед кластеризацией были исключены вакансии с длиной лемматизированного текста менее 200 символов (менее 1% наблюдений).

**Интерпретация кластеров**

Анализ топ-термов центроидов позволил выделить следующие типы вакансий:

🔹 Кластер 1 — Строительно-технические позиции

Характерные термины: объект, технический, документация, строительный.
Медианная зарплата: 150 000 руб.
Кластер характеризуется высокой долей вакансий с высоким уровнем оплаты труда.

🔹 Кластер 0 — Коммерческие позиции / продажи

Термины: продажа, клиент, переговоры, менеджер.
Медианная зарплата: 147 000 руб.
Высокая концентрация высокооплачиваемых вакансий.

🔹 Кластер 5 — Проектная деятельность / IT

Термины: разработка, проект, команда, задача.
Медианная зарплата: 140 000 руб.

🔹 Кластер 9 — Бухгалтерия и учёт

Термины: бухгалтерский, учёт, налоговый.
Медианная зарплата: 120 000 руб.

🔹 Кластер 4 — Сфера обслуживания / общепит

Термины: смена, гость, заказ, питание.
Медианная зарплата: 100 000 руб.

🔹 Кластер 3 — Административные функции

Термины: документ, подготовка, организация.
Медианная зарплата: 100 000 руб.

🔹 Кластер 8 — Вакансии с маркетингово-ориентированным стилем описания

Термины: наш, мы, ты, человек, работать.
Медианная зарплата: 100 000 руб.

Данный кластер объединяет вакансии, в описании которых преобладают общие и коммуникативные формулировки, ориентированные на привлечение кандидатов, а не на детальное описание профессиональных обязанностей.


🔹 Кластер 7 — Медицинские позиции

Термины: пациент, медицинский, клиника.
Медианная зарплата: 100 000 руб.

🔹 Кластер 2 — Рабочие позиции / логистика

Термины: день, автомобиль, место, месяц.
Медианная зарплата: 95 700 руб.

🔹 Кластер 6 — Частичная занятость / продавцы

Термины: неполный рабочий, продавец.
Медианная зарплата: 78 300 руб.
Практически полностью состоит из вакансий с низким уровнем оплаты труда.

**Связь кластеров с уровнем заработной платы**

Несмотря на то что кластеризация проводилась без использования переменной заработной платы, полученные кластеры демонстрируют различия в распределении уровней оплаты труда.

Кластеры строительных, коммерческих и проектных вакансий характеризуются наибольшими медианными значениями зарплаты и высокой долей high-вакансий.

Напротив, кластер частичной занятости практически полностью состоит из low-вакансий.

**Связь с результатами NLP-анализа**

Результаты кластеризации согласуются с ранее проведённым NLP-анализом:

В высокооплачиваемых кластерах доминируют управленческие, проектные и коммерческие термины (управление, продажи, проект, строительство).

В низкооплачиваемых кластерах преобладают операционные и сервисные функции (кассовые операции, сменная работа, обслуживание).

Таким образом, различия, выявленные при анализе отдельных текстовых признаков (TF-IDF и χ²), подтверждаются на уровне естественной структуры вакансий.

Это свидетельствует о согласованности результатов разных методов анализа и устойчивости выявленных закономерностей.

In [9]:
top_terms_df.head(10)

,cluster,count,top_terms
0,0,1506,"продажа, клиент, переговоры, база, менеджер, к..."
1,1,1247,"объект, технический, документация, строительны..."
2,2,2701,"день, автомобиль, место, месяц, плата, рабочий..."
3,3,2094,"документ, подготовка, организация, ведение, зн..."
4,4,1326,"смена, гость, заказ, час, питание, бесплатный,..."
5,5,1767,"разработка, проект, команда, понимание, задача..."
6,6,114,"неполный рабочий, неполный, отдых, продавец ка..."
7,7,838,"пациент, медицинский, клиника, специальность, ..."
8,8,1931,"наш, клиент, мы, что, команда, который, если, ..."
9,9,482,"бухгалтерский, учёт, бухгалтер, налоговый, док..."
